マウスの総移動距離を算出

import

In [20]:
import os
import math

関数の定義

In [40]:
def calculate_total_time(file_path):
    """
    指定されたファイルからタスクにかかった全体の時間を計算する
    """
    first_time = None
    last_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # 特殊ケース: MENU_ITEM_NAVIGATED で "C:\Users" を含む場合
                if line.startswith("MENU_ITEM_NAVIGATED") and "C:\\Users" in line:
                    parts = line.strip().rsplit(":", 1)
                    current_time = int(parts[-1].strip())
                else:
                    # 通常ケース: 2個目のコロンの後が時間
                    parts = line.strip().split(":")
                    if len(parts) > 2:
                        current_time = int(parts[2].strip())
                    else:
                        continue

                if first_time is None:
                    first_time = current_time
                last_time = current_time
            except Exception as e:
                print(f"Error processing line: {line}. Error: {e}")

    total_time = last_time - first_time if first_time is not None and last_time is not None else 0
    return total_time

def calculate_average_speed(file_path):
    """
    指定されたファイルからマウスの平均速度を計算する
    """
    total_time = 0.0
    total_distance = 0.0
    last_position = None
    last_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # マウスの移動に関する行を抽出
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    x, y = map(int, parts[-1].strip().strip("()").split(","))
                    current_position = (x, y)
                    current_time = int(parts[-2].strip())

                    if last_position is not None and last_time is not None:
                        # 前回の位置からの距離と経過時間を計算
                        distance = math.sqrt((current_position[0] - last_position[0])**2 +
                                             (current_position[1] - last_position[1])**2)
                        time_elapsed = current_time - last_time

                        total_distance += distance
                        total_time += time_elapsed

                    last_position = current_position
                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")

    average_speed = total_distance / total_time if total_time > 0 else 0
    return average_speed

def calculate_average_X_direction_speed(file_path):
    """
    指定されたファイルからマウスのX方向の平均速度を計算する

    Parameters:
        file_path (str): ログファイルのパス

    Returns:
        float: X方向の平均速度
    """
    total_time = 0.0
    total_x_distance = 0.0
    last_x = None
    last_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # マウスの移動に関する行を抽出
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    x, _ = map(int, parts[-1].strip().strip("()").split(","))  # X座標のみ取得
                    current_x = x
                    current_time = int(parts[-2].strip())  # タイムスタンプ

                    if last_x is not None and last_time is not None:
                        # X方向の移動距離と経過時間を計算
                        x_distance = abs(current_x - last_x)  # X方向の移動量
                        time_elapsed = current_time - last_time

                        total_x_distance += x_distance
                        total_time += time_elapsed

                    last_x = current_x
                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")

    average_x_speed = total_x_distance / total_time if total_time > 0 else 0
    return average_x_speed

def calculate_average_Y_direction_speed(file_path):
    """
    指定されたファイルからマウスのY方向の平均速度を計算する

    Parameters:
        file_path (str): ログファイルのパス

    Returns:
        float: Y方向の平均速度
    """
    total_time = 0.0
    total_y_distance = 0.0
    last_y = None
    last_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # マウスの移動に関する行を抽出
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    # 座標を取得
                    _, _, current_time_str, coordinates_str = parts
                    y = int(coordinates_str.strip().strip("()").split(",")[1].strip())  # Y座標のみ取得
                    current_y = y
                    current_time = int(current_time_str.strip())  # タイムスタンプ

                    if last_y is not None and last_time is not None:
                        # Y方向の移動距離と経過時間を計算
                        y_distance = abs(current_y - last_y)  # Y方向の移動量
                        time_elapsed = current_time - last_time

                        total_y_distance += y_distance
                        total_time += time_elapsed

                    last_y = current_y
                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line.strip()}. Error: {e}")

    average_y_speed = total_y_distance / total_time if total_time > 0 else 0
    return average_y_speed

def calculate_max_speed(file_path):
    """
    指定されたファイルからマウスの最大速度を計算する
    """
    max_speed = 0.0
    last_position = None
    last_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # マウスの移動に関する行を抽出
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    x, y = map(int, parts[-1].strip().strip("()").split(","))
                    current_position = (x, y)
                    current_time = int(parts[-2].strip())

                    if last_position is not None and last_time is not None:
                        # 前回の位置からの距離と経過時間を計算
                        distance = math.sqrt((current_position[0] - last_position[0])**2 +
                                             (current_position[1] - last_position[1])**2)
                        time_elapsed = current_time - last_time

                        if time_elapsed > 0:
                            # 現在の速度を計算
                            speed = distance / time_elapsed
                            # 最大速度を更新
                            max_speed = max(max_speed, speed)

                    last_position = current_position
                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")

    return max_speed

def calculate_average_acceleration(file_path):
    """
    指定されたファイルからマウスの平均加速度を計算する
    """
    velocities = []
    last_position = None
    last_time = None
    first_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    x, y = map(int, parts[-1].strip().strip("()").split(","))
                    current_position = (x, y)
                    current_time = int(parts[-2].strip())

                    if first_time is None:
                        first_time = current_time

                    if last_position is not None and last_time is not None:
                        # 距離と時間差から速度を計算
                        distance = math.sqrt((current_position[0] - last_position[0])**2 +
                                             (current_position[1] - last_position[1])**2)
                        time_elapsed = current_time - last_time
                        if time_elapsed > 0:
                            velocity = distance / time_elapsed
                            velocities.append(velocity)

                    last_position = current_position
                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")

    accelerations = []
    for i in range(1, len(velocities)):
        acceleration = (velocities[i] - velocities[i-1]) / (first_time if first_time else 1)
        accelerations.append(acceleration)

    average_acceleration = sum(accelerations) / len(accelerations) if accelerations else 0
    return average_acceleration

def calculate_average_absolute_acceleration(file_path):
    """
    指定されたファイルからマウスの加速度の絶対値の平均を計算する
    """
    velocities = []
    last_position = None
    last_time = None
    first_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    x, y = map(int, parts[-1].strip().strip("()").split(","))
                    current_position = (x, y)
                    current_time = int(parts[-2].strip())

                    if first_time is None:
                        first_time = current_time

                    if last_position is not None and last_time is not None:
                        # 距離と時間差から速度を計算
                        distance = math.sqrt((current_position[0] - last_position[0])**2 +
                                             (current_position[1] - last_position[1])**2)
                        time_elapsed = current_time - last_time
                        if time_elapsed > 0:
                            velocity = distance / time_elapsed
                            velocities.append(velocity)

                    last_position = current_position
                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")

    absolute_accelerations = []
    for i in range(1, len(velocities)):
        acceleration = abs(velocities[i] - velocities[i-1]) / (first_time if first_time else 1)
        absolute_accelerations.append(acceleration)

    average_absolute_acceleration = sum(absolute_accelerations) / len(absolute_accelerations) if absolute_accelerations else 0
    return average_absolute_acceleration

def avg_mouse_X_accel(file_path):
    """
    指定されたファイルからマウスのX方向の平均加速度の絶対値を計算する

    Parameters:
        file_path (str): ログファイルのパス

    Returns:
        float: X方向の平均加速度の絶対値
    """
    velocities = []  # X方向の速度を格納
    last_x = None
    last_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    x = int(parts[-1].strip().strip("()").split(",")[0])  # X座標のみ取得
                    current_x = x
                    current_time = int(parts[-2].strip())  # タイムスタンプ

                    if last_x is not None and last_time is not None:
                        # X方向の速度を計算
                        x_distance = current_x - last_x  # X方向の移動距離
                        time_elapsed = current_time - last_time

                        if time_elapsed > 0:
                            velocity = x_distance / time_elapsed
                            velocities.append(velocity)

                    last_x = current_x
                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")

    # X方向の加速度（絶対値）を計算
    absolute_accelerations = []
    for i in range(1, len(velocities)):
        acceleration = abs(velocities[i] - velocities[i-1])  # 加速度の絶対値
        absolute_accelerations.append(acceleration)

    # 平均加速度の絶対値を計算
    average_absolute_acceleration = (
        sum(absolute_accelerations) / len(absolute_accelerations) if absolute_accelerations else 0
    )
    return average_absolute_acceleration

def calculate_max_acceleration(file_path):
    """
    指定されたファイルからマウスの最大加速度を計算する
    """
    velocities = []
    last_position = None
    last_time = None
    first_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    x, y = map(int, parts[-1].strip().strip("()").split(","))
                    current_position = (x, y)
                    current_time = int(parts[-2].strip())

                    if first_time is None:
                        first_time = current_time

                    if last_position is not None and last_time is not None:
                        # 距離と時間差から速度を計算
                        distance = math.sqrt((current_position[0] - last_position[0])**2 +
                                             (current_position[1] - last_position[1])**2)
                        time_elapsed = current_time - last_time
                        if time_elapsed > 0:
                            velocity = distance / time_elapsed
                            velocities.append(velocity)

                    last_position = current_position
                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")

    accelerations = []
    for i in range(1, len(velocities)):
        acceleration = (velocities[i] - velocities[i-1]) / (last_time - first_time if last_time and first_time else 1)
        accelerations.append(acceleration)

    max_acceleration = max(accelerations) if accelerations else 0
    return max_acceleration

def calculate_min_acceleration(file_path):
    """
    指定されたファイルからマウスの最小加速度を計算する
    """
    velocities = []
    last_position = None
    last_time = None
    first_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    x, y = map(int, parts[-1].strip().strip("()").split(","))
                    current_position = (x, y)
                    current_time = int(parts[-2].strip())

                    if first_time is None:
                        first_time = current_time

                    if last_position is not None and last_time is not None:
                        # 距離と時間差から速度を計算
                        distance = math.sqrt((current_position[0] - last_position[0])**2 +
                                             (current_position[1] - last_position[1])**2)
                        time_elapsed = current_time - last_time
                        if time_elapsed > 0:
                            velocity = distance / time_elapsed
                            velocities.append(velocity)

                    last_position = current_position
                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")

    accelerations = []
    for i in range(1, len(velocities)):
        acceleration = (velocities[i] - velocities[i-1]) / (last_time - first_time if last_time and first_time else 1)
        accelerations.append(acceleration)

    min_acceleration = min(accelerations) if accelerations else 0
    return min_acceleration

def calculate_distance(file_path):
    """
    指定されたファイルからマウスの総移動距離を計算する
    """
    total_distance = 0.0
    last_position = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # マウスの移動に関する行を抽出
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    x, y = map(int, parts[-1].strip().strip("()").split(","))
                    current_position = (x, y)

                    if last_position is not None:
                        # 前回の位置からの距離を計算
                        distance = math.sqrt((current_position[0] - last_position[0])**2 +
                                             (current_position[1] - last_position[1])**2)
                        total_distance += distance

                    last_position = current_position
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")
    return total_distance

def calculate_pause_over_threshold(file_path, threshold=150):
    """
    指定されたファイルから一時停止が閾値（ミリ秒）を超えた回数を計算する
    """
    pause_count = 0
    last_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    current_time = int(parts[-2].strip())

                    if last_time is not None:
                        time_elapsed = current_time - last_time
                        if time_elapsed > threshold:
                            pause_count += 1

                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")

    return pause_count

def calculate_average_pause_time(file_path, threshold=150):
    """
    指定されたファイルから一時停止が閾値（ミリ秒）を超えた平均時間を計算する
    """
    pause_times = []
    last_time = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    current_time = int(parts[-2].strip())

                    if last_time is not None:
                        time_elapsed = current_time - last_time
                        if time_elapsed > threshold:
                            pause_times.append(time_elapsed)

                    last_time = current_time
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")

    average_pause_time = sum(pause_times) / len(pause_times) if pause_times else 0
    return average_pause_time

def calculate_action_pause_time(file_path, actions=("BUTTON_PRESSED", "MENU_ITEM_SELECTED")):
    """
    指定されたファイルからアクション後の一時停止時間の平均を算出する

    Parameters:
        file_path (str): 読み込むファイルのパス
        actions (tuple): 一時停止を計算する対象のアクション

    Returns:
        float: アクション後の一時停止時間の平均
    """
    action_times = []
    pause_times = []

    # ファイルを読み取る
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # アクション行を特定する
                for action in actions:
                    if action in line:
                        parts = line.strip().split(":")
                        timestamp = int(parts[-1].strip())
                        action_times.append(timestamp)
                        break
            except ValueError:
                print(f"Error processing line: {line}")
                continue

    # 一時停止時間を計算する
    for i in range(1, len(action_times)):
        pause_time = action_times[i] - action_times[i - 1]
        pause_times.append(pause_time)

    # 平均を計算する
    average_pause_time = sum(pause_times) / len(pause_times) if pause_times else 0
    return average_pause_time

def calculate_average_mouse_pause_time(file_path, mouse_events=("MOUSE_MOVED", "MOUSE_DRAGGED")):
    """
    指定されたファイルから連続するマウスイベント間の平均一時停止時間を算出する

    Parameters:
        file_path (str): ログファイルのパス
        mouse_events (tuple): 対象となるマウスイベントの種類 (デフォルト: MOUSE_MOVED と MOUSE_DRAGGED)

    Returns:
        float: 平均一時停止時間（ミリ秒単位）
    """
    mouse_timestamps = []
    last_event_type = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # 対象イベントの行を特定
                for event in mouse_events:
                    if event in line:
                        parts = line.strip().split(":")
                        if len(parts) >= 4:  # 行が期待される形式であることを確認
                            timestamp = int(parts[2].strip())  # 3番目のコロン後がタイムスタンプ
                            # 前回のイベントと同じ種類の場合のみタイムスタンプを追加
                            if last_event_type == event or last_event_type is None:
                                mouse_timestamps.append(timestamp)
                            last_event_type = event
                        break
            except ValueError as e:
                print(f"calculate_average_mouse_pause_time_Error processing line: {line.strip()} - Error: {e}")
                continue

    # 一時停止時間を計算
    if len(mouse_timestamps) < 2:
        return 0  # マウスイベントが1つ以下の場合は平均一時停止時間を0とする

    pause_times = [
        mouse_timestamps[i] - mouse_timestamps[i - 1]
        for i in range(1, len(mouse_timestamps))
    ]

    # 平均を計算
    average_pause_time = sum(pause_times) / len(pause_times)
    return average_pause_time

def calculate_episodes_count(file_path, mouse_events=("MOUSE_MOVED", "MOUSE_DRAGGED"), threshold=150):
    """
    指定されたファイルから連続するマウス移動のエピソード数を算出する

    Parameters:
        file_path (str): ログファイルのパス
        mouse_events (tuple): マウス移動とみなすイベント名 (デフォルト: "MOUSE_MOVED", "MOUSE_DRAGGED")
        threshold (int): エピソードを区切る一時停止時間の閾値 (デフォルト: 150ミリ秒)

    Returns:
        int: マウス移動エピソードの数
    """
    mouse_timestamps = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # マウスイベントの行を特定
                if any(event in line for event in mouse_events):
                    parts = line.strip().split(":")
                    if len(parts) >= 3:  # 行が期待される形式であることを確認
                        timestamp = int(parts[2].strip())  # 3番目の部分がタイムスタンプ
                        mouse_timestamps.append(timestamp)
            except ValueError as e:
                print(f"calculate_episodes_count_Error processing line: {line.strip()} - Error: {e}")
                continue

    if not mouse_timestamps:
        return 0  # マウスイベントがなければエピソード数は0

    episodes_count = 1  # 少なくとも1つのエピソードがあると仮定
    for i in range(1, len(mouse_timestamps)):
        # 隣接するタイムスタンプ間の差を計算
        if mouse_timestamps[i] - mouse_timestamps[i - 1] >= threshold:
            episodes_count += 1  # 閾値以上の一時停止があれば新しいエピソードとしてカウント

    return episodes_count


def calculate_average_key_pressed_pause_time(file_path, key_event="KEY_PRESSED", threshold=150):
    """
    指定されたファイルから連続する KEY_PRESSED イベント間の一時停止が
    閾値（ミリ秒）を超えた場合のみ、その平均一時停止時間を算出する

    Parameters:
        file_path (str): ログファイルのパス
        key_event (str): 対象となるイベント名 (デフォルト: "KEY_PRESSED")
        threshold (int): 一時停止時間の閾値（ミリ秒単位、デフォルト: 150）

    Returns:
        float: 平均一時停止時間（閾値を超える場合のみ、ミリ秒単位）
    """
    key_timestamps = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # KEY_PRESSED イベントの行を特定
                if key_event in line:
                    parts = line.strip().split(":")
                    if len(parts) >= 3:  # 行が期待される形式であることを確認
                        timestamp = int(parts[-1].strip())  # 最後の部分がタイムスタンプ
                        key_timestamps.append(timestamp)
            except ValueError as e:
                print(f"calculate_average_key_pressed_pause_time_Error processing line: {line.strip()} - Error: {e}")
                continue

    # 一時停止時間を計算
    if len(key_timestamps) < 2:
        return 0  # KEY_PRESSED イベントが1つ以下の場合は平均一時停止時間を0とする

    # 閾値を超えた一時停止時間のみをリストに追加
    pause_times = [
        key_timestamps[i] - key_timestamps[i - 1]
        for i in range(1, len(key_timestamps))
        if key_timestamps[i] - key_timestamps[i - 1] > threshold
    ]

    # 平均を計算
    average_pause_time = sum(pause_times) / len(pause_times) if pause_times else 0
    return average_pause_time

def direction_changes_count(file_path, mouse_events=("MOUSE_MOVED", "MOUSE_DRAGGED")):
    """
    指定されたファイルから移動方向の変化回数を算出する

    Parameters:
        file_path (str): ログファイルのパス
        mouse_events (tuple): マウス移動とみなすイベント名 (デフォルト: "MOUSE_MOVED", "MOUSE_DRAGGED")

    Returns:
        int: マウス移動方向の変化回数
    """
    prev_x, prev_y = None, None
    prev_dx, prev_dy = None, None
    direction_changes = 0

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # マウスイベントの行を特定
                if any(event in line for event in mouse_events):
                    parts = line.strip().split(":")
                    if len(parts) >= 4:  # 行が期待される形式であることを確認
                        coordinates = parts[-1].strip().strip("()").split(",")
                        x, y = int(coordinates[0]), int(coordinates[1])

                        if prev_x is not None and prev_y is not None:
                            dx = x - prev_x
                            dy = y - prev_y

                            # 移動方向の変化を検出 (符号の変化を判定)
                            if (prev_dx is not None and prev_dy is not None) and (
                                (dx != 0 and dx * prev_dx < 0) or
                                (dy != 0 and dy * prev_dy < 0)
                            ):
                                direction_changes += 1

                            # 現在の移動量を保存
                            prev_dx, prev_dy = dx, dy

                        # 現在の座標を保存
                        prev_x, prev_y = x, y

            except ValueError as e:
                print(f"direction_changes_count_Error processing line: {line.strip()} - Error: {e}")
                continue

    return direction_changes

def calculate_total_angle_changes(file_path, mouse_events=("MOUSE_MOVED", "MOUSE_DRAGGED")):
    """
    指定されたファイルから連続するマウス移動間の移動方向角度の変化の合計を算出する

    Parameters:
        file_path (str): ログファイルのパス
        mouse_events (tuple): マウス移動とみなすイベント名 (デフォルト: "MOUSE_MOVED", "MOUSE_DRAGGED")

    Returns:
        float: 移動方向角度の変化の合計（度数単位）
    """
    coordinates = []
    
    # ファイルから座標を抽出
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # マウスイベントの行を特定
                if any(event in line for event in mouse_events):
                    parts = line.strip().split(":")
                    if len(parts) >= 4:  # 行が期待される形式であることを確認
                        coord_parts = parts[-1].strip().strip("()").split(",")
                        x, y = int(coord_parts[0]), int(coord_parts[1])
                        coordinates.append((x, y))
            except ValueError as e:
                print(f"calculate_total_angle_changes_Error processing line: {line.strip()} - Error: {e}")
                continue

    if len(coordinates) < 3:
        return 0.0  # 少なくとも2つの移動（3点）が必要

    total_angle_change = 0.0

    # 角度の変化を計算
    for i in range(1, len(coordinates) - 1):
        x1, y1 = coordinates[i - 1]
        x2, y2 = coordinates[i]
        x3, y3 = coordinates[i + 1]

        # ベクトル1とベクトル2を計算
        v1 = (x2 - x1, y2 - y1)
        v2 = (x3 - x2, y3 - y2)

        # ベクトルの長さ
        norm_v1 = math.sqrt(v1[0]**2 + v1[1]**2)
        norm_v2 = math.sqrt(v2[0]**2 + v2[1]**2)

        if norm_v1 == 0 or norm_v2 == 0:
            continue  # 移動がない場合はスキップ

        # 内積
        dot_product = v1[0] * v2[0] + v1[1] * v2[1]
        cos_theta = max(min(dot_product / (norm_v1 * norm_v2), 1), -1)  # 数値誤差対策

        # 角度を計算 (ラジアンから度に変換)
        angle_change = math.degrees(math.acos(cos_theta))
        total_angle_change += angle_change

    return total_angle_change

def process_logs(base_folder_path, output_file):
    """
    各フォルダのログファイルを処理し、総移動距離、平均速度、全体の時間、平均アクション一時停止時間、
    平均マウス一時停止時間、平均キー押下一時停止時間、マウス移動エピソード数、方向変化回数、
    角度変化の合計、X方向とY方向の平均速度などを計算して保存する
    """
    results = []

    for i in range(1, 26):  # フォルダ番号の範囲を指定（例: 1〜25）
        subject_number = f"{i:02d}"
        folder_path = os.path.join(base_folder_path, subject_number, "logs")

        if os.path.exists(folder_path):
            log_files = [file for file in os.listdir(folder_path) if file.startswith("ui-log") and file.endswith(".txt")]
            # 日付の昇順でソート
            log_files.sort()

            for index, file_name in enumerate(log_files):
                file_path = os.path.join(folder_path, file_name)
                total_distance = calculate_distance(file_path)
                average_speed = calculate_average_speed(file_path)
                max_speed = calculate_max_speed(file_path)
                total_time = calculate_total_time(file_path)
                pause_count = calculate_pause_over_threshold(file_path)
                average_pause_time = calculate_average_pause_time(file_path)
                average_acceleration = calculate_average_acceleration(file_path)
                average_absolute_acceleration = calculate_average_absolute_acceleration(file_path)
                max_acceleration = calculate_max_acceleration(file_path)
                min_acceleration = calculate_min_acceleration(file_path)
                action_pause_time = calculate_action_pause_time(file_path)
                mouse_pause_time = calculate_average_mouse_pause_time(file_path)
                key_pause_time = calculate_average_key_pressed_pause_time(file_path)
                episodes_count = calculate_episodes_count(file_path)
                direction_changes = direction_changes_count(file_path)
                total_angle_changes = calculate_total_angle_changes(file_path)
                average_x_speed = calculate_average_X_direction_speed(file_path)
                average_y_speed = calculate_average_Y_direction_speed(file_path)
                average_x_accel = avg_mouse_X_accel(file_path)

                # Taskの決定
                if index == 0:
                    task = "training"
                else:
                    task = f"task_{index:02d}"

                results.append((
                    subject_number, file_name, task, total_distance, average_speed, max_speed, 
                    total_time, pause_count, average_pause_time, average_acceleration, 
                    average_absolute_acceleration, max_acceleration, min_acceleration, 
                    action_pause_time, mouse_pause_time, key_pause_time, episodes_count, 
                    direction_changes, total_angle_changes, average_x_speed, average_y_speed,
                    average_x_accel
                ))

    # 結果をファイルに保存
    with open(output_file, 'w', encoding='utf-8') as output:
        output.write("Subject,File,Task,Total Distance,Average Speed,Max Speed,Total Time,Pause Count,Average Pause Time,Average Acceleration,Average Absolute Acceleration,Max Acceleration,Min Acceleration,Action Pause Time,Mouse Pause Time,Key Pause Time,Episodes Count,Direction Changes,Total Angle Changes,Average X Direction Speed,Average Y Direction Speed, Average X Accel\n")
        for (subject, file_name, task, distance, speed, max_speed, time, pause_count, 
             avg_pause_time, acceleration, absolute_acceleration, max_acc, min_acc, 
             action_pause_time, mouse_pause_time, key_pause_time, episodes_count, 
             direction_changes, total_angle_changes, average_x_speed, average_y_speed,
             average_x_accel) in results:
            output.write(f"{subject},{file_name},{task},{distance:.2f},{speed:.2f},{max_speed:.2f},{time},{pause_count},{avg_pause_time:.2f},{acceleration:.2f},{absolute_acceleration:.2f},{max_acc:.2f},{min_acc:.2f},{action_pause_time:.2f},{mouse_pause_time:.2f},{key_pause_time:.2f},{episodes_count},{direction_changes},{total_angle_changes:.2f},{average_x_speed:.2f},{average_y_speed:.2f},{average_x_accel:.2f}\n")

    print(f"結果を {output_file} に保存しました。")

In [41]:
# ベースフォルダのパスと出力ファイル名を指定
base_folder_path = r".\SubjectData"
output_file = "ui_log_features.csv"

process_logs(base_folder_path, output_file)


結果を ui_log_features.csv に保存しました。
